In [26]:
import os
import sys
from pathlib import Path


def initialize_environment():
    BASE_DIR = str(Path("./../").resolve())
    if BASE_DIR not in sys.path:
        sys.path.append(BASE_DIR)


initialize_environment()
from analytics.strategies.sma_crossovers import SMAStrategy, Trend
from analytics.services.alpha_vantage import TimeInterval, AVTimeseries


def print_heading(text: str):
    print("###############################\n")
    print(text)
    print("\n###############################")


%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [27]:
# define the symbol for which you need to test the SMA crossover strategy. 
# TODO: use search endpoint after integration to discover the right symbol. 
# alpha vantage documentation - https://www.alphavantage.co/documentation/

symbol = "IBM"
# if you dont have an alpha vantage api_key get it from - 
# https://www.alphavantage.co/support/#api-key
av_obj = AVTimeseries(api_key="your_api_key_here")



<IPython.core.display.Javascript object>

In [28]:
# Lets grab 60min intraday data for IBM
# supported time_intervals are ['ONE_MIN', 'FIVE_MIN', 'FIFTEEN_MIN', 'THIRTY_MIN', 'SIXTY_MIN']

av_intraday_df = av_obj.get_intraday_data(symbol="IBM", interval=TimeInterval.SIXTY_MIN)
display(av_intraday_df.head())

,open,high,low,close,volume
2020-09-22 15:00:00,120.13,120.5080,120.12,120.37,330986.0
2020-09-22 16:00:00,120.37,120.8099,120.29,120.52,506209.0
2020-09-22 17:00:00,120.51,120.5100,120.40,120.51,62054.0
2020-09-22 18:00:00,120.35,120.3500,120.35,120.35,200.0
2020-09-22 20:00:00,120.30,120.3000,120.30,120.30,1111.0


<IPython.core.display.Javascript object>

In [29]:
# Similarly lets grab daily data for IBM using Alpha Vantage

av_daily_df = av_obj.get_daily_data(symbol="IBM")
display(av_daily_df.head())

,open,high,low,close,volume
2020-05-13,119.95,119.99,114.85,115.73,5882800.0
2020-05-14,114.57,117.09,111.81,116.95,5259400.0
2020-05-15,115.93,117.39,115.25,116.98,4786300.0
2020-05-18,119.88,122.36,119.75,121.56,4207600.0
2020-05-19,121.19,122.17,120.14,120.29,3241100.0


<IPython.core.display.Javascript object>

In [30]:
# Bottom up approach - Lets us evaluate how profitable 20-10 SMA cross-over strategy for IBM is. 
# It is same simple as pasisng the correct dataframe and defining the slower moving average time period 
# and faster moving time period. 
# SO lets get to it. 

# Ensure that slow_ma is greater than fast_ma. It makes logical sense. 
# If slow_ma < fast_ma, there will be an AssertionError
SMAStrategy.evaluate_sma_crossover(raw_scrip_df=av_intraday_df, slow_ma=20, fast_ma=10, capture_trend=Trend.BULLISH)


,,percent_returns,session_details
sma_session_20_10,label_20_10,,
1,bullish,-1.321806,2020-09-23 12:00:00-2020-09-24 06:00:00
3,bullish,0.993737,2020-09-28 08:00:00-2020-09-29 16:00:00
5,bullish,-0.554220,2020-09-30 13:00:00-2020-10-01 13:00:00
7,bullish,-0.380921,2020-10-02 18:00:00-2020-10-02 20:00:00


<IPython.core.display.Javascript object>

### Top-to-Bottom approach 

* The SMAStrategy class provides helper functions to compute and evaluate numbers step by step. 

In [36]:
# define slow and fast moving average look back periods
# Again, ensure that slow_ma > fast_ma
slow_ma = 20
fast_ma = 10

# create an instance of SMAStrategy
sma_strategy_obj = SMAStrategy(scrip_df=av_intraday_df, slow_ma=20, fast_ma=10)


# compute simple moving averages
sma_df = sma_strategy_obj.compute_sma(look_back_periods=[10, 20])
print_heading("Simple Moving Averages")
display(sma_df)


# we can club the computation of SMA and annotattion using the sma_sessions method as well
sma_sessions_annotated_df = sma_strategy_obj.sma_sessions()
print_heading("Simple Moving Average Cross Over Strategy Annotated Sessions")
display(sma_sessions_annotated_df)

# use annotated data to get the summary of returns
column_suffix = f"{slow_ma}_{fast_ma}"
summary_df = sma_sessions_annotated_df.groupby(
    [f"sma_session_{column_suffix}", f"label_{column_suffix}"], as_index=True
).apply(SMAStrategy.compute_returns, slow_ma=slow_ma, fast_ma=fast_ma)
print_heading("Returns Summary")
# Note that the summary_df results will match the one from evaluate_sma_crossover classmathod.
display(summary_df)

###############################

Simple Moving Averages

###############################


,sma_10,sma_20
2020-09-22 15:00:00,120.37000,120.370000
2020-09-22 16:00:00,120.52000,120.520000
2020-09-22 17:00:00,120.51000,120.510000
2020-09-22 18:00:00,120.35000,120.350000
2020-09-22 20:00:00,120.30000,120.300000
...,...,...
2020-10-02 15:00:00,120.19773,120.742365
2020-10-02 16:00:00,120.32373,120.674365
2020-10-02 17:00:00,120.47273,120.625865
2020-10-02 18:00:00,120.55773,120.555365


###############################

Simple Moving Average Cross Over Strategy Annotated Sessions

###############################


,open,high,low,close,volume,sma_20,sma_10,sma_signal_20_10,sma_session_20_10,label_20_10
2020-09-22 15:00:00,120.13,120.5080,120.12,120.37,330986.0,120.370000,120.37000,False,0,bearish
2020-09-22 16:00:00,120.37,120.8099,120.29,120.52,506209.0,120.520000,120.52000,False,0,bearish
2020-09-22 17:00:00,120.51,120.5100,120.40,120.51,62054.0,120.510000,120.51000,False,0,bearish
2020-09-22 18:00:00,120.35,120.3500,120.35,120.35,200.0,120.350000,120.35000,False,0,bearish
2020-09-22 20:00:00,120.30,120.3000,120.30,120.30,1111.0,120.300000,120.30000,False,0,bearish
...,...,...,...,...,...,...,...,...,...,...
2020-10-02 15:00:00,121.04,121.5331,121.03,121.18,226659.0,120.742365,120.19773,False,6,bearish
2020-10-02 16:00:00,121.15,121.4000,120.49,120.56,597238.0,120.674365,120.32373,False,6,bearish
2020-10-02 17:00:00,120.57,120.9900,120.57,120.99,96646.0,120.625865,120.47273,False,6,bearish
2020-10-02 18:00:00,120.76,120.7600,120.30,120.30,1206.0,120.555365,120.55773,True,7,bullish


###############################

Returns Summary

###############################


,,percent_returns,session_details
sma_session_20_10,label_20_10,,
0,bearish,-0.116173,2020-09-22 15:00:00-2020-09-23 11:00:00
1,bullish,-1.321806,2020-09-23 12:00:00-2020-09-24 06:00:00
2,bearish,0.631313,2020-09-24 08:00:00-2020-09-25 20:00:00
3,bullish,0.993737,2020-09-28 08:00:00-2020-09-29 16:00:00
4,bearish,-1.136271,2020-09-29 17:00:00-2020-09-30 12:00:00
5,bullish,-0.554220,2020-09-30 13:00:00-2020-10-01 13:00:00
6,bearish,0.315170,2020-10-01 14:00:00-2020-10-02 17:00:00
7,bullish,-0.380921,2020-10-02 18:00:00-2020-10-02 20:00:00


<IPython.core.display.Javascript object>